In [1]:
!pip install gensim
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 24.0/24.0 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 58.6/58.6 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 5.3 MB/s eta 0:00:00


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 50.5/50.5 KB 856.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein
  Running setup.py install for python-Levenshtein: started
  Running setup.py install for python-Levenshtein: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [28 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\Levenshtein
  copying Levenshtein\StringMatcher.py -> build\lib.win-amd64-3.8\Levenshtein
  copying Levenshtein\__init__.py -> build\lib.win-amd64-3.8\Levenshtein
  running egg_info
  writing python_Levenshtein.egg-info\PKG-INFO
  writing dependency_links to python_Levenshtein.egg-info\dependency_links.txt
  deleting python_Levenshtein.egg-info\entry_points.txt
  writing namespace_packages to python_Levenshtein.egg-info\namespace_packages.txt
  writing requirements to python_Levenshtein.egg-info\requires.txt
  writing top-level names to python_Levenshtein.egg-info\top_level.txt
  reading manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'CO

In [105]:
import numpy as np

In [106]:
import gensim
import pandas as pd

In [3]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Reading and Exploring the Dataset
The dataset we are using here is a subset of Amazon reviews from the Cell Phones & Accessories category. The data is stored as a JSON file and can be read using pandas.

Link to the Dataset: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

In [4]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json.gz", lines=True)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


In [5]:
df.shape

(194439, 9)

### Simple Preprocessing & Tokenization
The first thing to do for any data science task is to clean the data.
For NLP, we apply various processing like converting all the words to lower case, trimming spaces, removing punctuations. 
This is something we will do over here too.

Additionally, we can also remove stop words like 'and', 'or', 'is', 'the', 'a', 'an' and convert words to their root forms like 'running' to 'run'.

In [35]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [36]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [37]:
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [9]:
df.reviewText.loc[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

### Phrases

In [24]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

In [38]:
phrase_model = Phrases(review_text, min_count=5, threshold=10, connector_words=ENGLISH_CONNECTOR_WORDS)

In [39]:
new_sentence = review_text
review_text=phrase_model[new_sentence]

In [83]:
review_text.corpus

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [27]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

### Training the Word2Vec Model

Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

#### Initialize the model

In [84]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=8, vector_size=100
)


#### Build Vocabulary

In [85]:
model.build_vocab(review_text, progress_per=1000)

In [86]:
len(model.wv)

50230

#### Train the Word2Vec Model

In [87]:
model.corpus_count

194439

In [88]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(56486590, 78604830)

### Save the Model

Save the model so that it can be reused in other applications

In [89]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

### Finding Similar Words and Similarity between words
https://radimrehurek.com/gensim/models/word2vec.html

In [90]:
model.wv.most_similar("awesome")

[('amazing', 0.9108415246009827),
 ('fantastic', 0.8008341789245605),
 ('wonderful', 0.7779650092124939),
 ('great', 0.7297639846801758),
 ('fabulous', 0.7046794891357422),
 ('awsome', 0.6784698367118835),
 ('outstanding', 0.6752285361289978),
 ('excellent', 0.6718266010284424),
 ('incredible', 0.6545916199684143),
 ('terrific', 0.6466891169548035)]

In [91]:
model.wv.similarity(w1="terrible", w2="awful")

0.82188374

In [92]:
model.wv.similarity(w1="great", w2="good")

0.76775014

In [93]:
model.wv.doesnt_match(['great',"terrible","awful"])

'great'

In [94]:
model.wv.get_vector('awesome')

array([-1.1164309 , -2.2439957 ,  0.16364455, -1.1073526 ,  0.09167603,
       -1.2306896 ,  1.4845383 , -2.4244783 , -1.4303428 , -0.09632602,
        0.15062463,  2.3404152 ,  2.3948748 ,  3.186282  , -1.619755  ,
       -1.7909261 ,  0.20144664,  0.23629843,  0.12376058,  0.2177282 ,
       -0.99215126,  0.78636456, -0.8695633 ,  0.38098148,  1.9721413 ,
       -2.5553634 ,  1.7134237 ,  0.78614956,  0.1650329 ,  4.0093946 ,
       -2.1987522 ,  2.9624007 ,  0.7253825 ,  1.0196636 , -1.8725603 ,
        2.2582378 , -0.36055818, -0.37801602,  1.3150245 ,  0.8041439 ,
       -1.670807  ,  1.2450826 ,  0.5078476 ,  1.520132  ,  2.6681225 ,
        2.0097244 , -0.54908836,  0.2045926 , -2.4189556 , -1.4250797 ,
        1.4909133 ,  1.7663994 , -1.3214839 , -1.0700272 , -2.4712825 ,
       -0.7672729 ,  0.23570529,  2.6563706 , -0.26527044, -1.4286234 ,
        3.6572464 , -1.7365708 , -1.027965  , -0.35986215, -2.8169394 ,
        0.33126634,  2.00924   , -3.5945952 , -3.660745  , -0.68

In [95]:
np.asarray(model.wv.vectors)

array([[-6.2229884e-01,  5.0729120e-01,  2.5551608e-01, ...,
        -1.4449873e+00,  2.2720318e+00, -5.6894958e-01],
       [-2.2012453e-01,  3.0129261e+00,  1.9596481e+00, ...,
        -2.8553966e-01,  2.4585009e-01,  6.4338110e-02],
       [ 1.4531115e+00, -8.4974319e-01,  1.7040898e+00, ...,
        -9.6949995e-01,  2.0040590e-01, -1.1053458e+00],
       ...,
       [-4.9601313e-02,  3.9993711e-02, -2.2285722e-02, ...,
        -2.7933925e-02,  2.7257010e-02,  1.8592365e-02],
       [ 2.1659428e-02, -8.7808473e-03, -2.7271293e-02, ...,
         1.9390217e-03, -1.0492652e-02, -1.7131561e-02],
       [-4.9496241e-02,  3.5520360e-02,  3.1805277e-02, ...,
         3.6624130e-02,  5.4555148e-02, -1.3306812e-02]], dtype=float32)

In [100]:
np.asarray(model.wv.index_to_key)[:30]

array(['the', 'it', 'and', 'to', 'is', 'this', 'for', 'of', 'my', 'that',
       'phone', 'on', 'in', 'with', 'you', 'case', 'but', 'have', 'not',
       'was', 'as', 'so', 'one', 'are', 'if', 'very', 'like', 'or', 'can',
       'great'], dtype='<U30')

In [101]:
model.wv.similar_by_vector(np.random.randint(100))

[('pretty_cool', 0.7357825636863708),
 ('cool', 0.7354509234428406),
 ('neat', 0.6881674528121948),
 ('good', 0.6873372197151184),
 ('great', 0.6718146204948425),
 ('attractive', 0.6250503063201904),
 ('distinctive', 0.6090043187141418),
 ('lovely', 0.6029127240180969),
 ('fantastic', 0.6013885140419006),
 ('classy', 0.6011518836021423)]

In [107]:
mol=gensim.models.Word2Vec.load('word2vec-amazon-cell-accessories-reviews-short.model')

In [108]:
mol.wv.most_similar("awesome")

[('amazing', 0.9108415246009827),
 ('fantastic', 0.8008341789245605),
 ('wonderful', 0.7779650092124939),
 ('great', 0.7297639846801758),
 ('fabulous', 0.7046794891357422),
 ('awsome', 0.6784698367118835),
 ('outstanding', 0.6752285361289978),
 ('excellent', 0.6718266010284424),
 ('incredible', 0.6545916199684143),
 ('terrific', 0.6466891169548035)]

In [109]:
mol.wv.get_index('awesome')

313

In [163]:
encoder=[]
for sentence in review_text.corpus[:50]:
    tokens=[]
    for word in sentence:
        try:
            tokens.append(mol.wv.get_index(word))
        except: pass
    encoder.append(tokens)

In [166]:
pd.Series(encoder)

0     [39, 132, 35, 2, 505, 35, 37, 81, 26, 0, 2556,...
1     [60, 1907, 76, 26, 0, 205, 565, 39, 69, 39, 50...
2     [60, 23, 313, 2, 149, 8, 10, 132, 21, 855, 17,...
3     [208, 522, 12, 29, 74, 2, 19, 12, 128, 1730, 1...
4     [313, 853, 11, 2, 107, 29, 28, 32, 96, 11, 748...
5     [60, 149, 98, 0, 363, 215, 100, 8, 762, 2, 142...
6     [156, 37, 20, 2083, 1, 115, 220, 15832, 2, 77,...
7     [1, 246, 6, 0, 104, 422, 105, 1, 63, 49, 8, 10...
8     [35, 15, 399, 1369, 320, 10, 36, 163, 13, 35, ...
9     [5, 4, 776, 15, 25, 855, 2, 320, 8, 10, 100, 5...
10    [5, 15, 122, 204, 11, 0, 2, 510, 56, 44006, 36...
11    [5, 4, 0, 104, 50, 15, 17, 45, 6, 8, 295, 0, 1...
12    [1927, 451, 20, 1512, 1, 25, 20231, 333, 2, 44...
13    [880, 153, 7, 0, 3342, 50, 112, 24790, 1139, 6...
14    [37, 79, 261, 261, 10, 15, 6, 699, 2, 8, 101, ...
15    [33, 53, 4, 65, 1339, 27, 351, 847, 99, 77, 24...
16    [1, 70, 29, 115, 1579, 61, 26, 1452, 26, 0, 58...
17    [1515, 5, 789, 478, 70, 37, 20, 41, 2, 37,